In [ ]:
!pip install yfinance


In [37]:
from datetime import datetime
import yfinance as yf
import time

In [3]:
class Mytradingstrategy:
    def __init__(self, name):
        self.__name = name
    
    def gerentae_signals(self,price_data):
        print("this method is indented to be over ridded")
        return "hold"
    
    @property
    def name(self):
        return self.__name

In [4]:
class MySMATradingStrategy(Mytradingstrategy):
    def __init__(self,swindow, lwindow):
        self.__swindow = swindow
        self.__lwindow = lwindow
        super().__init__('my sma trading strategy')
    
    def gerentae_signals(self,price_data):
        if len(price_data[-self.__swindow:]) < self.__swindow:
            return "hold"
        
        short_avg = sum(price_data[-self.__swindow:])/self.__swindow
        long_avg = sum(price_data[-self.__lwindow:])/self.__lwindow

        if short_avg > long_avg:
            return "buy"
        elif short_avg < long_avg:
            return "sell"
        else:
            return "hold"
            

    @property
    def swindow(self):
        return self.__swindow
    
    @property
    def lwindow(self):
        return self.__lwindow

In [5]:
class Mytrade:
    def __init__ (self, strategy_name , signal, amount):
        self.__strategy_name = strategy_name
        self.__signal = signal
        self.__amount = amount
        self.__timestamp = datetime.now()
    

    def execute(self):
        print(f"executing {self.__signal} trade at {self.__amount} with {self.__strategy_name} strategy at {self.__timestamp}")

    
    @property
    def strategy_name(self):
        return self.__strategy_name
    
    @property
    def signal(self):
        return self.__signal

    @property
    def amount(self):
        return self.__amount
    
    @property
    def timestamp(self):
        return self.__timestamp

In [12]:
#Mock Trading API
class MocktradeAPI:
    def __init__(self,balance):
        self.__balance = balance

    def place_order(self,trade,price):
        if trade.signal == "buy" and price * trade.amount >= self.__balance:
            self.__balance -= price * trade.amount
            print(f"buying {trade.amount} at {price} with {trade.strategy_name} strategy")
        elif trade.signal == "sell" and price * trade.amount <= self.__balance:
            self.__balance += price * trade.amount
            print(f"selling {trade.amount} at {price} with {trade.strategy_name} strategy")
        else:
            print("not enough balance")
            
    @property
    def get_balance(self):
        return self.__balance

In [29]:
class Mytradingsystem:
    def __init__(self,api,strategy, symbol):
        self.__api = api
        self.__strategy = strategy
        self.__symbol = symbol
        self.__price_data = []
    
    def fetch_price_Data(self):
        data = yf.download(self.__symbol, period='1d', interval='1m')
        if not data.empty:
            self.__price_data.append(data['Close'].iloc[-1])
            if len(self.__price_data) > self.__strategy.lwindow:
                self.__price_data.pop(0)
                print(f'fetched new data{self.__price_data}')
        else:
            print("no data fetched")
    
    def run (self):
        self.fetch_price_Data()
        signal = self.__strategy.gerentae_signals(self.__price_data)
        print(f"signal generated: {signal}")
        if signal in ['buy','sell']:
            trade = Mytrade(self.__strategy.name, signal, 1)
            trade.execute()
            self.__api.place_order(trade, self.__price_data[-1])
    
    @property
    def api(self):
        return self.__api
    
    @property
    def strategy(self):
        return self.__strategy
    
    @property
    def symbol(self):
        return self.__symbol
    
    @property
    def price_data(self):
        return self.__price_data


In [39]:
symbol = 'AAPL'
api = MocktradeAPI(1000)
strategy = MySMATradingStrategy(5, 20)
tradingsystem = Mytradingsystem(api, strategy, symbol)

for i in range(3):
    tradingsystem.run()
    print(f"balance: {api.get_balance}")
    time.sleep(3)

[*********************100%%**********************]  1 of 1 completed


signal generated: hold
balance: 1000


[*********************100%%**********************]  1 of 1 completed


signal generated: hold
balance: 1000


[*********************100%%**********************]  1 of 1 completed


signal generated: hold
balance: 1000
